# Training Sequential Circuits with Tensorflow

This notebook shows the expands upon the sequential circuits structure
and optimize the circuit using Tensorflow. <br>
Currently Tensorflow only supports 2 expectation values, which is only allows for multi-class classification of 4 classes. <br> 
This is due to this error https://github.com/PennyLaneAI/pennylane/issues/937.

In [41]:
#!pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
from model import Conv1DLayer, PoolingLayer, unitarity_conv1d, unitarity_pool, unitarity_toffoli
import tensorflow as tf

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

# Defining the device

In [20]:
#Specify how many wires will be used
num_wires = 12
num_ancilla = 4
num_work_wires = num_wires - num_ancilla

#Embedding size
embedding_size = 2**num_work_wires
dummy_embedding = [1] * embedding_size

#Specify what type of device you are using.
device_type = 'default.qubit'
#device_type = 'lightning.qubit'

#Initialize Device
dev = qml.device(device_type, wires=num_wires) 

# Defining circuit to be trained

In [21]:
def circuit(num_wires, num_ancilla):
    @qml.qnode(dev, interface='tf', diff_method ='adjoint')
    def func(inputs, params): 
        work_wires = list(range(num_wires - num_ancilla))
        ancilla_wires = list(range(len(work_wires),num_wires))

        # Data Embedding Layer
        AmplitudeEmbedding(inputs, wires=work_wires, normalize=True)

        # Hidden Layers
        work_wires, params = Conv1DLayer(unitarity_conv1d , 15)(work_wires , params)
        work_wires, params = PoolingLayer(unitarity_pool , 2)(work_wires , params)
        work_wires, params = Conv1DLayer(unitarity_conv1d , 15)(work_wires , params)
        work_wires, params = PoolingLayer(unitarity_pool , 2)(work_wires , params)

        toffoli_wires = [1,7,8]
        qml.Toffoli(wires=toffoli_wires)
        qml.PauliX(toffoli_wires[0])
        toffoli_wires[-1] += 1

        qml.Toffoli(wires=toffoli_wires)
        qml.PauliX(toffoli_wires[0])
        qml.PauliX(toffoli_wires[1])
        toffoli_wires[-1] += 1

        qml.Toffoli(wires=toffoli_wires)
        qml.PauliX(toffoli_wires[0])
        toffoli_wires[-1] += 1

        qml.Toffoli(wires=toffoli_wires)

        return [qml.expval(qml.PauliZ(wire)) for wire in ancilla_wires]
    return func

# Import training Data

In [22]:
import tensorflow as tf
labels = [0,1,2,3,4,5,6,7]

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0  # normalize the data

x_train_filter = [i for i, y in enumerate(y_train) if y in labels]
x_test_filter =  [i for i, y in enumerate(y_test) if y in labels]

X_train, X_test = x_train[x_train_filter], x_test[x_test_filter]
Y_train, Y_test = y_train[x_train_filter], y_test[x_test_filter]

X_train = tf.image.resize(X_train[:], (embedding_size, 1)).numpy()
X_test = tf.image.resize(X_test[:], (embedding_size, 1)).numpy()
X_train, X_test = tf.squeeze(X_train).numpy(), tf.squeeze(X_test).numpy()


# Preparing the circuit for Tensorflow

In [35]:
params_shapes = {"params": 357}
qlayer = qml.qnn.KerasLayer(circuit(num_wires, num_ancilla),params_shapes,output_dim=4)
output = tf.keras.layers.Dense(1,activation='softmax',dtype='float32')

In [36]:
model = tf.keras.models.Sequential([qlayer,output])

In [37]:
opt = tf.keras.optimizers.SGD(learning_rate=0.5)

In [38]:
params = tf.Variable(tf.convert_to_tensor(np.random.normal(size = 357)))

In [39]:
model.compile(opt, loss='categorical_crossentropy',metrics=['accuracy'])

# Training Model

In [40]:
model.fit(X_train, Y_train, epochs=8, batch_size=32)

Epoch 1/8


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:Mul]